pca_w2v_150features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_w2v/w2v_withstopwordchangelevels_150_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,5.719816,3.753577,-7.270468,-6.683765,-2.429075,1.618299,-5.832024,-2.125934,-1.753451,...,-1.328400e-08,1.600552e-07,3.656112e-07,-1.258745e-08,-6.595957e-08,8.024780e-08,-1.015240e-07,1.652006e-07,5.482539e-09,4.632340e-08
1,1,17.746299,-13.330659,-9.330557,1.143561,-1.791711,4.704435,-2.671869,3.445507,-5.427289,...,-6.000920e-09,3.735713e-08,-2.350416e-07,1.218062e-08,-9.182038e-08,2.500201e-07,1.289237e-07,-5.152249e-08,-6.788565e-08,5.411288e-08
2,2,-2.226274,1.718558,-2.493961,0.407887,-0.487651,-0.096254,2.220553,0.864598,-0.678770,...,-3.324168e-08,1.154838e-07,3.844717e-09,1.408011e-07,-1.446384e-08,-7.978240e-08,-1.742395e-07,-2.444011e-08,-7.276425e-08,1.078987e-07
3,3,-11.682953,2.388827,1.861568,-1.603650,-2.558248,1.741855,-4.619755,2.287277,1.682215,...,-3.479320e-08,2.182697e-07,7.724274e-08,-1.965868e-07,1.005511e-08,7.704525e-08,-2.705252e-07,1.175403e-07,1.636219e-07,-3.528624e-07
4,4,-4.725348,1.352140,-8.217170,-0.507472,0.542053,-8.566001,3.370496,-2.786307,4.873884,...,-7.991142e-08,6.000083e-08,-2.809470e-07,9.844024e-09,-2.478792e-07,-1.749872e-07,-8.114737e-08,-4.760530e-08,2.542054e-07,8.378963e-08


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,5.719816,3.753577,-7.270468,-6.683765,-2.429075,1.618299,-5.832024,-2.125934,-1.753451,-1.195281,...,-1.328400e-08,1.600552e-07,3.656112e-07,-1.258745e-08,-6.595957e-08,8.024780e-08,-1.015240e-07,1.652006e-07,5.482539e-09,4.632340e-08
1,17.746299,-13.330659,-9.330557,1.143561,-1.791711,4.704435,-2.671869,3.445507,-5.427289,-1.576950,...,-6.000920e-09,3.735713e-08,-2.350416e-07,1.218062e-08,-9.182038e-08,2.500201e-07,1.289237e-07,-5.152249e-08,-6.788565e-08,5.411288e-08
2,-2.226274,1.718558,-2.493961,0.407887,-0.487651,-0.096254,2.220553,0.864598,-0.678770,3.692568,...,-3.324168e-08,1.154838e-07,3.844717e-09,1.408011e-07,-1.446384e-08,-7.978240e-08,-1.742395e-07,-2.444011e-08,-7.276425e-08,1.078987e-07
3,-11.682953,2.388827,1.861568,-1.603650,-2.558248,1.741855,-4.619755,2.287277,1.682215,-0.036047,...,-3.479320e-08,2.182697e-07,7.724274e-08,-1.965868e-07,1.005511e-08,7.704525e-08,-2.705252e-07,1.175403e-07,1.636219e-07,-3.528624e-07
4,-4.725348,1.352140,-8.217170,-0.507472,0.542053,-8.566001,3.370496,-2.786307,4.873884,-0.821005,...,-7.991142e-08,6.000083e-08,-2.809470e-07,9.844024e-09,-2.478792e-07,-1.749872e-07,-8.114737e-08,-4.760530e-08,2.542054e-07,8.378963e-08


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.361979283834055


**Model Building**

**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.5,0.5,0.550579,0.497501,0.324427,0.324427,0.37284,0.327415


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2222    0.0637    0.0990       157
           2     0.4619    0.8945    0.6092       237
           3     0.2500    0.0385    0.0667       130

    accuracy                         0.4332       524
   macro avg     0.3114    0.3322    0.2583       524
weighted avg     0.3375    0.4332    0.3217       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.5,0.5,0.550579,0.497501,0.324427,0.324427,0.372840,0.327415
1,RandomForest,1.0,1.0,1.000000,1.000000,0.433206,0.433206,0.337506,0.321738


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:

classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5190    0.3407    0.4114       361
           2     0.5303    0.7288    0.6139       553
           3     0.5426    0.3954    0.4575       306

    accuracy                         0.5303      1220
   macro avg     0.5306    0.4883    0.4942      1220
weighted avg     0.5300    0.5303    0.5147      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2430    0.1656    0.1970       157
           2     0.4295    0.5781    0.4928       237
           3     0.2245    0.1692    0.1930       130

    accuracy                         0.3531       524
   macro avg     0.2990    0.3043    0.2943       524
weighted avg     0.3227    0.3531    0.3298       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.519672,0.519672,0.529591,0.489414,0.429389,0.429389,0.403164,0.389851
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.361621,0.328493
2,Adaboost,0.530328,0.530328,0.530021,0.514717,0.353053,0.353053,0.322742,0.329784


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9486    0.8172    0.8780       361
           2     0.8116    0.9892    0.8916       553
           3     0.9702    0.7451    0.8429       306

    accuracy                         0.8770      1220
   macro avg     0.9101    0.8505    0.8708      1220
weighted avg     0.8919    0.8770    0.8754      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3535    0.2229    0.2734       157
           2     0.4790    0.7215    0.5758       237
           3     0.2500    0.1308    0.1717       130

    accuracy                         0.4256       524
   macro avg     0.3608    0.3584    0.3403       524
weighted avg     0.3846    0.4256    0.3849       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.519672,0.519672,0.529591,0.489414,0.429389,0.429389,0.403164,0.389851
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.361621,0.328493
2,Adaboost,0.530328,0.530328,0.530021,0.514717,0.353053,0.353053,0.322742,0.329784
3,GBM,0.877049,0.877049,0.891895,0.875352,0.425573,0.425573,0.384592,0.384938


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3996    0.5346    0.4573       361
           2     0.5435    0.3165    0.4000       553
           3     0.3422    0.4641    0.3939       306

    accuracy                         0.4180      1220
   macro avg     0.4284    0.4384    0.4171      1220
weighted avg     0.4504    0.4180    0.4154      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2821    0.3503    0.3125       157
           2     0.4526    0.2616    0.3316       237
           3     0.2448    0.3615    0.2919       130

    accuracy                         0.3130       524
   macro avg     0.3265    0.3245    0.3120       524
weighted avg     0.3499    0.3130    0.3160       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.500000,0.500000,0.550579,0.497501,0.324427,0.324427,0.372840,0.327415
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.433206,0.433206,0.337506,0.321738
2,SVM,0.418033,0.418033,0.450408,0.415438,0.312977,0.312977,0.349925,0.316012


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5263    0.6094    0.5648       361
           2     0.6122    0.7450    0.6721       553
           3     0.5581    0.2353    0.3310       306

    accuracy                         0.5770      1220
   macro avg     0.5655    0.5299    0.5227      1220
weighted avg     0.5732    0.5770    0.5548      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3005    0.3503    0.3235       157
           2     0.4498    0.5485    0.4943       237
           3     0.3077    0.1231    0.1758       130

    accuracy                         0.3836       524
   macro avg     0.3527    0.3406    0.3312       524
weighted avg     0.3698    0.3836    0.3641       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.519672,0.519672,0.529591,0.489414,0.429389,0.429389,0.403164,0.389851
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.361621,0.328493
2,Adaboost,0.530328,0.530328,0.530021,0.514717,0.353053,0.353053,0.322742,0.329784
3,GBM,0.877049,0.877049,0.891895,0.875352,0.425573,0.425573,0.384592,0.384938
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.577049,0.577049,0.573220,0.554814,0.383588,0.383588,0.369837,0.364121


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3239    0.7618    0.4545       361
           2     0.5420    0.2333    0.3262       553
           3     0.3910    0.1699    0.2369       306

    accuracy                         0.3738      1220
   macro avg     0.4190    0.3883    0.3392      1220
weighted avg     0.4396    0.3738    0.3418      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2926    0.6561    0.4047       157
           2     0.4080    0.2152    0.2818       237
           3     0.2979    0.1077    0.1582       130

    accuracy                         0.3206       524
   macro avg     0.3328    0.3263    0.2816       524
weighted avg     0.3461    0.3206    0.2879       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.519672,0.519672,0.529591,0.489414,0.429389,0.429389,0.403164,0.389851
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.419847,0.419847,0.361621,0.328493
2,Adaboost,0.530328,0.530328,0.530021,0.514717,0.353053,0.353053,0.322742,0.329784
3,GBM,0.877049,0.877049,0.891895,0.875352,0.425573,0.425573,0.384592,0.384938
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.577049,0.577049,0.573220,0.554814,0.383588,0.383588,0.369837,0.364121
6,GNB,0.373770,0.373770,0.439595,0.341766,0.320611,0.320611,0.346106,0.287947
